# Solving problems by Searching
This notebook serves as supporting material for topics covered in Solving Problems by Searching 

## Problem 
Let's see how we define the Problem. 

As discussed in the lecture, `Problem` class has five methods.

* `initial_state(self)` : This returns the initial state 

* `actions(self, state)` : This method returns all the possible actions agent can execute in the given state `state`.

* `result(self, state, action)` : This returns the resulting state if action `action` is taken in the state `state`. This `Problem` class only deals with deterministic outcomes. So we know for sure what every action in a state would result to.

* `goal_test(self, state)` : Return a boolean for a given state - `True` if it is a goal state, else `False`.

* `path_cost(self, c, state1, action, state2)` : Return the cost of the path that arrives at `state2` as a result of taking `action` from `state1`, assuming total cost of `c` to get up to `state1`.

In this implementation we will combine 

Run the next cell to see how the Problem is defined

In [1]:
class GraphProblem():
    
    def __init__(self, vertices, edges, start, goal, heuristic_values=[]):
        self.vertices = vertices
        self.edges = edges
        self.start = start
        self.goal = goal
        self.heuristic_values = heuristic_values
            
    def initial_state(self):
      return self.start

    def goal_test(self, state):
      return state == self.goal
      
    def result(self, state):    
        connected_actions = []
        for edge in self.edges:
            if edge[0] == state:
                connected_actions.append((edge[1], edge[1]))  # (action, resulting state)
            elif edge[1] == state:
                connected_actions.append((edge[0], edge[0]))  # (action, resulting state)
        connected_actions.sort()
        return connected_actions
        
    def path_cost(self, state, newstate):
        for edge in self.edges:
            if (edge[0] == state and edge[1] == newstate) or (edge[1] == state and edge[0] == newstate):
                return edge[2]  # Return the cost associated with the edge
        return float('inf')  # Return infinity if no edge exists between the states

    def heuristic(self, state):
        return self.heuristic_values[self.vertices.index(state)]
    
    

## Node
Let's see how we define a Node. Run the next cell to see how class Node is defined. The Node class helps in constructing paths from the goal state back to the initial state. By maintaining references to parent nodes, we can traverse the path in reverse order, effectively reconstructing the sequence of actions leading to the goal state. 

In [2]:
class Node():
    def __init__(self, state, parent, action, gcost=0, hcost=0):
        self.state = state
        self.parent = parent # parent of the current node
        self.action = action # action that was taken to reach this node
        self.gcost = gcost # cost
        self.hcost = hcost # heuristic
    def __lt__(self, other):
        return self.gcost+self.hcost < other.gcost+other.hcost

## Datastructure for Frontier

In [3]:
import heapq
# Last in first out frontier
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier.pop()
            return node

# First in first out frontier
class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier.pop(0)
            return node
class PQueueFrontier(StackFrontier):
    def __init__(self):
        self.frontier = []

    def add(self, node):
        heapq.heappush(self.frontier, node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = heapq.heappop(self.frontier)
            return node

## Graph Search Algorithm

In [4]:
def graph_search(problem):
    # Initialize frontier to just the starting position
    start = Node(state=problem.initial_state(), parent=None, action=None)
    frontier = Frontier();
    frontier.add(start)
    
    # Initialize an empty explored set
    explored = set()
    
    # Keep looping until solution found
    while True:
        # If nothing left in frontier, then no path
        if frontier.empty():
            raise Exception("no solution")

        # Choose a node from the frontier
        node = frontier.remove()

        # Mark node as explored
        explored.add(node.state)
        
        
        # If node is the goal, then we have a solution
        if problem.goal_test(node.state):
            actions = []
            cells = []
            while node.parent is not None:
                actions.append(node.action)
                cells.append(node.state)
                node = node.parent
            
            actions.append(node.action)
            cells.append(node.state)
            
            actions.reverse()
            cells.reverse()
            solution = (actions, cells)
            return solution, explored

        # Add neighbors to frontier
        for action, state in problem.result(node.state):
            if not frontier.contains_state(state) and state not in explored:
                child = Node(state=state, parent=node, action=action)
                frontier.add(child)
                

## Graph Search Algorithm (Uniformed Cost, A* and Greedy)

In [30]:
def graph_search_cost(problem, algo):
    
    # Initialize frontier to just the starting position
    start = Node(state=problem.start, parent=None, action=None, gcost=0, hcost=0)
        
    frontier = PQueueFrontier()
    frontier.add(start)
    
    # Initialize an empty explored set
    explored = set()
    
    # Keep looping until solution found
    while True:
        # If nothing left in frontier, then no path
        if frontier.empty():
            raise Exception("no solution")
        
        # Choose a node from the frontier
        node = frontier.remove()

        # Mark node as explored
        explored.add(node.state)
        
        # If node is the goal, then we have a solution
        if problem.goal_test(node.state):
            actions = []
            cells = []
            while node.parent is not None:
                actions.append(node.action)
                cells.append(node.state)
                node = node.parent
            actions.append(node.action)
            cells.append(node.state[0])
            
            actions.reverse()
            cells.reverse()
            solution = (actions, cells)
            return solution, explored

        
        # Add neighbors to frontier
        for action, state in problem.result(node.state):
            #if not frontier.contains_state(state) and state not in explored:
            if state not in explored:
                if algo == 'ucs':
                    child = Node(state=state, parent=node, action=action, gcost=node.gcost+problem.path_cost(node.state, state), hcost=0)
                elif algo == 'greedy':
                    child = Node(state=state, parent=node, action=action, gcost=0, hcost=problem.heuristic(state))
                elif algo == 'a*':
                    child = Node(state=state, parent=node, action=action, gcost=node.gcost+problem.path_cost(node.state, state), hcost=problem.heuristic(state))
                else:
                    raise Exception("Error")
                   
                frontier.add(child)

## Solution

In [16]:
# Problem 1
vertices = ['S', 'A', 'B', 'C', 'D', 'E', 'F', 'H', 'J', 'K', 'L', 'T']
edges = [
    ('S', 'A'), ('S', 'B'),
    ('A', 'F'),
    ('B', 'F'), ('B', 'C'),
    ('C', 'E'),
    ('D', 'H'),
    ('C', 'J'),
    ('F', 'H'),
    ('H', 'K'),
    ('J', 'L'),
    ('K', 'J'), ('K', 'T'),
    ('L', 'T')
]

problem = GraphProblem(vertices, edges, 'S', 'J')
solution, explored = graph_search(problem)
print(solution)
print(explored)

([None, 'B', 'C', 'J'], ['S', 'B', 'C', 'J'])
{'A', 'J', 'C', 'B', 'S', 'E', 'H', 'F'}


In [35]:
# Problem 1, with cost
vertices = ['S', 'A', 'B', 'C', 'D', 'E', 'F', 'H', 'J', 'K', 'L', 'T']
heuristics =[10,  6,   20,   2,   5,   1,  2,  2,   3,   1,   4,   0]
edges = [
    ('S', 'A', 2), ('S', 'B', 10),
    ('A', 'F', 4),
    ('B', 'F', 2), ('B', 'C', 5),
    ('C', 'E', 5),
    ('D', 'H', 3),
    ('C', 'J', 12),
    ('F', 'H', 8),
    ('H', 'K', 5),
    ('J', 'L', 4),
    ('K', 'J', 1), ('K', 'T', 7),
    ('L', 'T', 5)
]
problem = GraphProblem(vertices, edges, 'S', 'T', heuristics)
solution, explored = graph_search_cost(problem, algo='astart')
print(solution)
print(explored)

([None, 'A', 'F', 'H', 'K', 'T'], ['S', 'A', 'F', 'H', 'K', 'T'])
['S', 'A', 'B', 'F', 'C', 'H', 'E', 'J', 'D', 'K', 'L', 'T']


In [8]:
# Problem 2
vertices =  ['S', 'A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'G']
node_values = [0, 60, 50, 55, 50, 56, 50, 39, 0, 0]
edges = [
    ('S', 'A', 100), ('S', 'B', 3), ('S', 'E', 14), ('S', 'F', 4),
    ('A', 'C', 4), ('B', 'D', 4), ('C', 'I', 50), ('D', 'I', 50), ('E', 'H', 16),
    ('F', 'H', 16), ('H', 'I', 30), ('I', 'G', 10)
]
problem = GraphProblem(vertices, edges, 'S', 'G')
solution, explored = graph_search(problem)
print(solution)
print(explored)

([None, 'F', 'H', 'I', 'G'], ['S', 'F', 'H', 'I', 'G'])
{'S', 'F', 'H', 'I', 'G'}


In [9]:
# Problem 2 UCS
vertices =  ['S', 'A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'G']
heuristic = [0, 60, 50, 55, 50, 56, 50, 39, 0, 0]
edges = [
    ('S', 'A', 100), ('S', 'B', 3), ('S', 'E', 14), ('S', 'F', 4),
    ('A', 'C', 4), ('B', 'D', 4), ('C', 'I', 50), ('D', 'I', 50), ('E', 'H', 16),
    ('F', 'H', 16), ('H', 'I', 30), ('I', 'G', 10)
]
problem = GraphProblem(vertices, edges, 'S', 'G', heuristic)
solution, explored = graph_search_cost(problem, algo='ucs')
print(solution)
print(explored)

([None, 'F', 'H', 'I', 'G'], ['S', 'F', 'H', 'I', 'G'])
{'S', 'F', 'E', 'H', 'I', 'G', 'D', 'B'}
